In [ ]:
import os, shutil

# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = 'metamaps'

# The directory where we will
# store our smaller dataset
base_dir = 'metamaps_input'

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
len(os.listdir(original_dataset_dir))

In [ ]:
os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

In [ ]:
import random

# Randomize the list of files
input_files = os.listdir(original_dataset_dir)
random.shuffle(input_files)

len(input_files)

In [ ]:
bucket_size = 8500

# Copy first 17000 metamaps images to train_dir
for fname in input_files[0:2*bucket_size]:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 8500 metamaps to validation_cats_dir
for fname in input_files[bucket_size*2:bucket_size*3]:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 8500 metamaps to test_cats_dir
for fname in input_files[bucket_size*3:bucket_size*4]:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(validation_dir)))
print(len(os.listdir(test_dir)))

In [ ]:
os.listdir('..')

In [ ]:
os.path.abspath('.')